# -----Please make appropriate bucket name and project id changes before proceeding------

## M5 Data Set
Unfortunately License forbids redistribution of M5 data set so go to kaggle and download

Should give you 5 files:

calendar.csv - Contains information about the dates on which the products are sold.
sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
sell_prices.csv - Contains information about the price of the products sold per store and date.
sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)
# Example transfer of files to a gcs bucket.
# Use gsutil to create bucket (note region where Vertex)
gsutil mb -l us-central1 -p retail1-vf gs://retail1-gcp-m5-vf
# Copy m5 files to bucket
gsutil cp -j *.csv gs://retail1-gcp-m5-vf

### Install Vertex AI SDK, Authenticate, and upload of a Dataset to your GCS bucket
After the SDK installation the kernel will be automatically restarted. You may see this error message Your session crashed for an unknown reason which is normal

In [ ]:
%%capture
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
 
import IPython
 
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enter your project and GCS bucket

Enter your Project ID in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
BUCKET_NAME = "gs://sk-forecasting/m5-kaggle-data" 
REGION = "us-central1"  

# Initialize Vertex AI SDK

Initialize the *client* for Vertex AI.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

### Load up the sales data which is in a wide format and put it in a long format using melt() functionality in python

In [ ]:
import numpy as np
import pandas as pd

train_sales = pd.read_csv('gs://sk-forecasting/m5-kaggle-data/sales_train_evaluation.csv') 

In [ ]:
train_sales.head(5)


In [ ]:
sales = pd.melt(train_sales, 
              id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
              var_name = 'd', 
              value_name = "sales")
del train_sales # Clean up some memory

In [ ]:
sales.head(5)

### Create a new dataset to put the tables into

In [ ]:
%%bigquery
CREATE SCHEMA `sk-ai-ml-poc.m5_forecasting` 

In [ ]:
sales.to_csv("gs://sk-forecasting/m5-kaggle-data/sales_melted.csv", index=False) 

### Load up data into BigQuery for easy joining

TODO: 
* Change BQ project and datasets to fit your environment
* Change GCS locations 

Following cells should look like this:
```
!bq load --autodetect \
    --source_format=CSV \
    m5.calendar \
    gs://lowes-vertex-forecast-poc/kaggle-data/calendar.csv
```

In [ ]:
!bq load --autodetect \
    --source_format=CSV \
    m5_forecasting.calendar \
    gs://sk-forecasting/m5-kaggle-data/calendar.csv

In [ ]:
!bq load --autodetect \
    --source_format=CSV \
    m5_forecasting.sell_prices \
    gs://sk-forecasting/m5-kaggle-data/sell_prices.csv

In [ ]:
!bq load --autodetect \
    --source_format=CSV \
    m5_forecasting.sales_melted \
    gs://sk-forecasting/m5-kaggle-data/sales_melted.csv

In [ ]:
PROJECT_ID='sk-ai-ml-poc'  # <---CHANGE THIS
BQ_DATASET="m5_forecasting"               # <---CHANGE THIS

print(PROJECT_ID)

## [1] Create Activity Tables

In [ ]:
%%bigquery

 CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.activity_all AS
 SELECT sal.item_id as product_id,
        sal.store_id as location_id,
        sal.sales as gross_quantity,
        cal.*,
        price.sell_price
   FROM `sk-ai-ml-poc.m5_forecasting.sales_melted` sal
 JOIN `sk-ai-ml-poc.m5_forecasting.calendar` cal ON sal.d = cal.d
 JOIN `sk-ai-ml-poc.m5_forecasting.sell_prices` price
   ON (price.item_id = sal.item_id AND 
       price.store_id = sal.store_id AND 
       price.wm_yr_wk = cal.wm_yr_wk)

### 1.b Smaller actitivty table for testing, `actitivty_fold1`

In [ ]:
 %%bigquery
 CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.activity_fold1
 AS (SELECT 
     table_a.product_id,
     table_a.location_id,
     table_a.gross_quantity,
     TIMESTAMP(table_a.date) as datetime,
     table_a.weekday,
     table_a.wday,
     table_a.month,
     table_a.year,
     table_a.event_name_1,
     table_a.event_type_1,
     table_a.event_name_2,
     table_a.event_type_2,
     table_a.snap_CA,
     table_a.snap_TX,
     table_a.snap_WI,
     table_a.sell_price,
     CASE 
         WHEN table_a.date BETWEEN '2015-01-01' AND '2016-01-17' THEN 'TRAIN'
         WHEN table_a.date BETWEEN '2016-01-18' AND '2016-02-28' THEN 'VALIDATE'
         WHEN table_a.date BETWEEN '2016-02-29' AND '2016-03-27' THEN 'TEST' 
     END AS ml_use
     FROM sk-ai-ml-poc.m5_forecasting.activity_all as table_a
     WHERE table_a.date BETWEEN '2015-01-01' AND '2016-03-27'
     )
 ;

## [2] Create Locations Table

In [ ]:
 %%bigquery 
 CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.locations
 AS (SELECT 
     DISTINCT store_id as location_id,
     state_id
     FROM sk-ai-ml-poc.m5_forecasting.sales_melted);

## [3] Create Products Table

In [ ]:
 %%bigquery 
 CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.products
 AS (SELECT 
     DISTINCT item_id as product_id,
     dept_id,
     cat_id
     FROM sk-ai-ml-poc.m5_forecasting.sales_melted);

## [4] Create Plan Table

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.plan_fold1
AS (SELECT 
    table_a.product_id,
    table_a.location_id,
    table_a.gross_quantity,
    TIMESTAMP(table_a.date) as datetime,
    table_a.weekday,
    table_a.wday,
    table_a.month,
    table_a.year,
    table_a.event_name_1,
    table_a.event_type_1,
    table_a.event_name_2,
    table_a.event_type_2,
    table_a.snap_CA,
    table_a.snap_TX,
    table_a.snap_WI,
    table_a.sell_price,
    'HOLDOUT' as ml_use
    FROM sk-ai-ml-poc.m5_forecasting.activity_all as table_a
    WHERE table_a.date BETWEEN '2016-03-28' AND '2016-04-24');
    -- NULL prediction target
    UPDATE sk-ai-ml-poc.m5_forecasting.plan_fold1
    SET gross_quantity = NULL WHERE datetime < '2016-04-25';

### 4.b Larger Plan Table


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE sk-ai-ml-poc.m5_forecasting.plan_table_all
AS (SELECT 
    table_a.product_id,
    table_a.location_id,
    table_a.gross_quantity,
    TIMESTAMP(table_a.date) as datetime,
    table_a.weekday,
    table_a.wday,
    table_a.month,
    table_a.year,
    table_a.event_name_1,
    table_a.event_type_1,
    table_a.event_name_2,
    table_a.event_type_2,
    table_a.snap_CA,
    table_a.snap_TX,
    table_a.snap_WI,
    table_a.sell_price,
    FROM sk-ai-ml-poc.m5_forecasting.activity_all as table_a
    WHERE table_a.date BETWEEN '2016-01-01' AND '2016-04-24');
    -- NULL prediction target
    UPDATE sk-ai-ml-poc.m5_forecasting.plan_table_all
    SET gross_quantity = NULL WHERE datetime >= '2016-03-29';

In [ ]:
import os

PROJECT_ID = "sk-ai-ml-poc" 

# END

Now all data tables are created in your BQ environment, proceed to the [Modeling Notebook]